<a href="https://colab.research.google.com/github/olgaselesnjova/Tweet-sentiment-analysis/blob/main/Tweet_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSOM 23SM1 ML Hometask 2

Kaggle competition  https://www.kaggle.com/competitions/gsom-23sm1-ml-hometask-2/leaderboard

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_palette('muted') 
sns.set_color_codes('muted')
sns.set_style('white') 

import warnings
warnings.filterwarnings('ignore')

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer # like Mapper

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import SGDClassifier

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import ParameterGrid

from tqdm import tqdm

In [ ]:
from xgboost import XGBClassifier
import re    # for regular expressions 
import nltk  # for text manipulation 
import string 
from nltk.stem.porter import *
import gensim#.models.doc2vec import LabeledSentence

In [ ]:
# finction for the cleaning process
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt 

In [ ]:
# for gensim.models.doc2vec
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v.wv[word].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary                                     
            continue
    if count != 0:
        vec /= count
    return vec

# 1. Basic preprocessing

## 1.1. Dataset

In [ ]:
data = pd.read_csv('corona_train.csv', encoding='ISO-8859-1', index_col=0)
data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
ID,,,,,,
0,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
1,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
2,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regions first confirmed COVID-1...,Positive
3,3807,48759,"Atlanta, GA USA",16-03-2020,Due to COVID-19 our retail store and classroom...,Positive
4,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy th...",Negative


In [ ]:
print(data.ndim)
print(data.shape)
print(data.size)

2
(26973, 6)
161838


## 1.2. Value types

In [ ]:
# converting to string
data.UserName = data.UserName.astype(str)
data.ScreenName = data.ScreenName.astype(str)

**Conclusion:** 

* `UserName` and `ScreenName` are unique.

* `Location` repeats.

## 1.3 Duplicates

In [ ]:
data.drop_duplicates(subset='OriginalTweet').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26973 entries, 0 to 26972
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       26973 non-null  object
 1   ScreenName     26973 non-null  object
 2   Location       21264 non-null  object
 3   TweetAt        26973 non-null  object
 4   OriginalTweet  26973 non-null  object
 5   Sentiment      26973 non-null  object
dtypes: object(6)
memory usage: 1.4+ MB


**Conclusion:** `OriginalTweet` don't have duplicates.

# 1.4. Missing values

In [ ]:
# percentage of missing data
(data.isna().sum() / len(data)).rename('percent of NA').to_frame().style.bar(vmax=1, vmin=0).format('{:.2%}')

,percent of NA
UserName,0.00%
ScreenName,0.00%
Location,21.17%
TweetAt,0.00%
OriginalTweet,0.00%
Sentiment,0.00%


# 2. EDA

## 2.1. `Sentiment` : target variable

In [ ]:
data.Sentiment.value_counts()

Positive              7446
Negative              6533
Neutral               4977
Extremely Positive    4349
Extremely Negative    3668
Name: Sentiment, dtype: int64

 **Note**: let's look at `Extremely Negative` and `Extremely Positive` tweets!

In [ ]:
data[data.Sentiment == 'Extremely Negative'].OriginalTweet.sample(1).iloc[0]

"Don't be panic in storing grocery, selfishness can create shortage of food and it may kill poor and daily wagers more than #Corona in the 3rd world country like Pakistan.  \n #CoronaInPakistan #CoronaVirusUpdate #CoronavirusOutbreak https://t.co/k6sA9wIol4"

In [ ]:
data[data.Sentiment == 'Extremely Positive'].OriginalTweet.sample(1).iloc[0]

'@mayorportman Wells Fargo is committed to helping customers experiencing hardships from COVID-19. If in need of assistance, customers can call 1-800-219-9739 to speak with a trained specialist about options available for their consumer lending, small busi'

## 2.2. `TweetAt`

In [ ]:
# for checking
data.TweetAt.loc[0]

Timestamp('2020-03-16 00:00:00')

In [ ]:
# converting to datetime
data.TweetAt = pd.to_datetime(data.TweetAt)

## 2.3. `Location`

In [ ]:
data.Location.value_counts()

United States                    351
London                           349
London, England                  337
New York, NY                     244
Washington, DC                   238
                                ... 
BANGKOK - THAILAND                 1
London UK / Ubatuba SP Brasil      1
Oshawa                             1
philly                             1
Haverford, PA                      1
Name: Location, Length: 8811, dtype: int64

In [ ]:
data[data.Location.notna()]\
[data[data.Location.notna()].Location.str.lower().str.contains('russi')]\
.Location.value_counts()

United States Russia Earth ?      4
Moscow, Russia                    3
Russia                            2
space? the Chinese? Russians??    1
Stavropol.Russia                  1
King of Prussia, PA               1
Russian Crypto City               1
Name: Location, dtype: int64

# 3. Feature Engineering

## 3.1. Splitting `TweetAt`

In [ ]:
data['month'] = data.TweetAt.dt.month
data['day'] = data.TweetAt.dt.day
data['dayofweek'] = data.TweetAt.dt.dayofweek
data['weekday'] = data.TweetAt.dt.weekday # binary

## 3.2. `tweetlength`

In [ ]:
data['tweetlength'] = data.OriginalTweet.str.len()

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26973 entries, 0 to 26972
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   UserName       26973 non-null  object        
 1   ScreenName     26973 non-null  object        
 2   Location       21264 non-null  object        
 3   TweetAt        26973 non-null  datetime64[ns]
 4   OriginalTweet  26973 non-null  object        
 5   Sentiment      26973 non-null  object        
 6   month          26973 non-null  int64         
 7   day            26973 non-null  int64         
 8   dayofweek      26973 non-null  int64         
 9   weekday        26973 non-null  int64         
 10  tweetlength    26973 non-null  int64         
 11  CleanTweet     26973 non-null  object        
dtypes: datetime64[ns](1), int64(5), object(6)
memory usage: 3.7+ MB


## 3.3. Cleaning : `CleanTweet`

**Note:** let's clean up `OriginalTweet` by getting rid of `#`, `@`, enters, `http\`. Moreover let's implement the `stemming` routine to make out of words more common base for the further analysis.

In [ ]:
# for checking 
data['OriginalTweet'][5]

"All month there hasn't been crowding in the supermarkets or restaurants, however reducing all the hours and closing the malls means everyone is now using the same entrance and dependent on a single supermarket. #manila #lockdown #covid2019 #Philippines https://t.co/HxWs9LAnF9"

In [ ]:
# for checking 
data['CleanTweet'][5]

'month there hasn been crowd supermarket restaur howev reduc hour close mall mean everyon use same entranc depend singl supermarket manila lockdown covid2019 philippin hxws9lanf9'

In [ ]:
data['CleanTweet'] = data['OriginalTweet'].str.lower()

data['CleanTweet'] = data['CleanTweet'].str.replace("[^a-zA-Z#]", " ") 
data['CleanTweet'] = data['CleanTweet'].apply(lambda x: x.replace('\n', ' ')) 

data['CleanTweet'] = data['CleanTweet'].str.replace('[^A-Za-z0-9]+',' ', regex=True) 
data['CleanTweet'] = data['CleanTweet'].str.replace(r"http\S+", "", regex=True) 

data['CleanTweet'] = np.vectorize(remove_pattern)(data['CleanTweet'], "@[\w]*") 
data['CleanTweet'] = data['CleanTweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [ ]:
# stemming
tokenized_tweets = data['CleanTweet'].apply(lambda x: x.split()) 
stemmer = PorterStemmer() 
tokenized_tweets = tokenized_tweets.apply(lambda x: [stemmer.stem(i) for i in x]) 

for i in range(len(tokenized_tweets)):
    tokenized_tweets[i] = ' '.join(tokenized_tweets[i])   
data['CleanTweet'] = tokenized_tweets

## 3.4. Feature selection

### `numeric_features`

In [ ]:
numeric_features = data.select_dtypes(include=np.number).columns.tolist()
numeric_features

['month', 'day', 'dayofweek', 'weekday', 'tweetlength']

### `Location`

### `CleanTweet`

### `features`

In [ ]:
features = numeric_features + ['Location', 'CleanTweet']

# 4. Train & Test Samples Splitting

In [ ]:
data[features].head(1)

,UserName,ScreenName,Location,CleanTweet
ID,,,,
0,3801,48753,Vagabonds,coronavirus australia woolworths to give elder...


In [ ]:
X, y = data[features], data.Sentiment

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, # Clean Tweet
                                                    y,
                                                    stratify=y,
                                                    test_size = 0.25,
                                                    random_state = 42)

# 5. Model

**Note:** since in the `pipeline` we will use `XGBClassifier()` that works only with numeric labels, it is necessary to convert 5-classes word labels to 5-classes numeric labels.

In [ ]:
y_train_num = y_train.replace({'Extremely Negative': 0,
                'Negative': 1,
                'Neutral': 2,
                'Positive': 3,
                'Extremely Positive': 4})

y_test_num = y_test.replace({'Extremely Negative': 0,
                'Negative': 1,
                'Neutral': 2,
                'Positive': 3,
                'Extremely Positive': 4})

y_num = y.replace({'Extremely Negative': 0,
                'Negative': 1,
                'Neutral': 2,
                'Positive': 3,
                'Extremely Positive': 4})


## 5.1. `Word2Vec()`

**Note:** we also tried:

* the classifier `gensim.models.Word2Vec()` for vectorization of the `CleanTweet`;

* the classifier `TfidfVectorizer()` for vectorization of the `CleanTweet`;

But scores are not so high, so let's use the following pipeline: `XGBClassifier()` & `CountVectorizer()`

# See 5.2. `XGBClassifier()` & `CountVectorizer()` (below)

### `Train Sample`

In [ ]:
X_train['CleanTweet'].reset_index().drop(columns = 'ID')['CleanTweet']

0        #vegan #coronaviru those eat them anim pay the...
1        wonder covid affect consum behavior media cons...
2        extend transit period johnson chanc lead today...
3        live govern urg peopl panic amid coronaviru cr...
4        mani face financi uncertainti thi time consum ...
                               ...                        
20224    #genz continu outspend their march amount even...
20225    look love milk market asset citi cannot contro...
20226    where covid kill over peopl peopl classifi wor...
20227    take time chat staff your local supermarket so...
20228    wait aussi slang come sanni hand sanitis self ...
Name: CleanTweet, Length: 20229, dtype: object

In [ ]:
?gensim.models.Word2Vec

In [ ]:
tokenized_tweets = X_train['CleanTweet'].reset_index().drop(columns = 'ID')['CleanTweet'].apply(lambda x: x.split()) # tokenizing 
model_w2v = gensim.models.Word2Vec(
            tokenized_tweets,
            vector_size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2,
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 2, # no.of cores
            seed = 34) 

In [ ]:
%%time
model_w2v.train(tokenized_tweets, total_examples= len(X_train['CleanTweet'].reset_index().drop(columns = 'ID')['CleanTweet']), epochs=20)

CPU times: user 1min 40s, sys: 321 ms, total: 1min 40s
Wall time: 50.8 s


(6382766, 8012000)

In [ ]:
wordvec_arrays = np.zeros((len(tokenized_tweets), 200)) 
for i in range(len(tokenized_tweets)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweets[i], 200)
    wordvec_df = pd.DataFrame(wordvec_arrays) 

wordvec_df.shape    

(20229, 200)

In [ ]:
wordvec_df

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.051751,-0.044537,-0.200707,-0.109736,0.038835,0.175783,-0.182812,-0.107739,0.273552,-0.064675,...,0.003784,-0.136183,-0.038762,-0.151570,0.057526,-0.114788,0.014622,-0.222910,0.160510,0.279523
1,0.050729,0.140039,-0.197878,-0.129984,-0.053340,0.067513,0.056197,-0.010796,0.236575,-0.125987,...,-0.015700,-0.061843,-0.112149,-0.071067,-0.122530,0.007717,-0.073251,-0.120119,0.071475,0.189200
2,0.101446,-0.108234,-0.119818,-0.042895,-0.075223,-0.155524,0.020671,-0.139043,0.256966,-0.068759,...,-0.101825,-0.158375,-0.099776,-0.030219,-0.031052,-0.071683,-0.081879,-0.305749,0.089141,0.357997
3,0.071717,-0.075007,-0.055861,-0.113304,-0.133429,0.036673,-0.201717,-0.139199,0.274253,-0.211267,...,-0.104364,-0.264682,0.029880,-0.231075,0.043712,-0.049515,-0.149536,-0.197162,0.146548,0.243187
4,0.051998,0.052723,-0.014037,-0.104180,-0.110245,0.024416,0.026521,0.033264,0.313415,0.004546,...,-0.024282,-0.096959,-0.130850,-0.097781,0.019408,0.039178,-0.121717,-0.119683,0.092316,0.230641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20224,-0.023177,0.120299,-0.020615,-0.090328,-0.094162,0.021291,-0.038620,-0.071031,0.305046,0.056699,...,0.046144,-0.147147,-0.159276,-0.125667,0.095521,-0.000382,-0.163799,-0.176154,0.040097,0.167940
20225,0.069573,-0.072607,0.006331,0.033339,-0.166438,0.117257,-0.047195,-0.057481,0.285601,-0.054671,...,0.028839,-0.154447,0.021021,-0.115833,0.049686,-0.103277,-0.172301,-0.176502,0.132982,0.351658
20226,0.037648,0.058374,0.093051,-0.144906,-0.090264,0.069592,-0.085424,-0.088590,0.275738,-0.049539,...,-0.057388,-0.224494,0.032029,-0.291356,0.124357,-0.091748,-0.071598,-0.350660,0.013561,0.396635
20227,0.015816,-0.108101,-0.122439,-0.100083,-0.123894,0.090127,-0.108031,-0.189202,0.343669,-0.026409,...,-0.019602,-0.253605,0.006311,-0.051872,0.065004,-0.188964,-0.147057,-0.208443,0.041304,0.357939


### `Test Sample`

In [ ]:
X_test['CleanTweet'].reset_index().drop(columns = 'ID')['CleanTweet']

0       rough weekend albert heijn supermarket restock...
1       #coronavirusoutbreak #cdc #covid groceri store...
2       dear sethi have thought bank employe home loan...
3       everyon worri about #coronaviru #toiletpap thi...
4       covid scam target deposit account rampant join...
                              ...                        
6739       groceri store #coronaviru tip point http suenv
6740    #paracetamol first line medicin cure #coronavi...
6741    covid wind turbin price increas renew renew en...
6742    way hacker scammer exploit panic espionag comm...
6743    instead hear more about selfish panick terribl...
Name: CleanTweet, Length: 6744, dtype: object

In [ ]:
tokenized_tweets = X_test['CleanTweet'].reset_index().drop(columns = 'ID')['CleanTweet'].apply(lambda x: x.split())

In [ ]:
wordvec_arrays = np.zeros((len(tokenized_tweets), 200)) 
for i in range(len(tokenized_tweets)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweets[i], 200)
    wordvec_df_test = pd.DataFrame(wordvec_arrays) 

wordvec_df_test.shape   

(6744, 200)

### Pipeline `XGBClassifier`

In [ ]:
pipeline = Pipeline(steps = [('XGBClassifier', XGBClassifier(max_depth=6, n_estimators=700, nthread= 3))])

In [ ]:
%%time
pipeline.fit(wordvec_df, y_train_num)

tf_pred = pipeline.predict(wordvec_df)
tf_pred_test = pipeline.predict(wordvec_df_test)

CPU times: user 1h 20min 20s, sys: 18.9 s, total: 1h 20min 39s
Wall time: 28min 47s


## 5.2. `XGBClassifier()` & `CountVectorizer()`

**Note:** 5.3. `GridSearch()` for `XGBClassifier()` & `CountVectorizer()` determined optimal parameters:

* xgb_learning_rate: 0.4
* xgb_max_depth: 5
* xgb_n_estimators: 600

In [ ]:
pipeline = Pipeline(steps = [('count_vec', CountVectorizer(stop_words="english", 
                                                           analyzer='word', 
                                                           ngram_range=(1, 3), 
                                                           max_df=1.0, 
                                                           min_df=1, 
                                                           max_features=None)),
                             
                            ('XGBClassifier', XGBClassifier(max_depth=5, 
                                                            n_estimators=600, 
                                                            learning_rate = 0.4, 
                                                            nthread= 3))])

In [ ]:
%%time
pipeline.fit(X_train.CleanTweet, y_train_num)

tf_pred_train = pipeline.predict(X_train.CleanTweet)
tf_pred_test = pipeline.predict(X_test.CleanTweet)

CPU times: user 24min 38s, sys: 13.2 s, total: 24min 51s
Wall time: 10min 3s


## 5.3. `GridSearch()` for `XGBClassifier()` & `CountVectorizer()`

In [ ]:
%%time
params_grid = dict(xgb_learning_rate = [0.2, 0.3, 0.4], n_estimators = [600, 700, 800])

results = []

for params in tqdm(ParameterGrid(params_grid)):
    
    pipeline = Pipeline(steps = [('count_vec', CountVectorizer(stop_words="english", 
                                                               analyzer='word', 
                                                               ngram_range=(1, 3), 
                                                               max_df=1.0, 
                                                               min_df=1, 
                                                               max_features=None)),

                                ('XGBClassifier', XGBClassifier(max_depth=5, 
                                                                n_estimators = params['n_estimators'], 
                                                                learning_rate = params['xgb_learning_rate'], 
                                                                nthread= 3))])
    
    
    
    
    pipeline.fit(X_train.CleanTweet, y_train_num)
    
    pipe_preds_train = pipeline.predict(X_train.CleanTweet)
    pipe_preds_test = pipeline.predict(X_test.CleanTweet)
    
    results.append(dict(
        
        params=params,
        
        precision_train = precision_score(y_true=y_train_num, y_pred=pipe_preds_train, average='macro'),
        precision_test = precision_score(y_true=y_test_num, y_pred=pipe_preds_test, average='macro'),       
        
        recall_train = recall_score(y_true=y_train_num, y_pred=pipe_preds_train, average='macro'),
        recall_test = recall_score(y_true=y_test_num, y_pred=pipe_preds_test, average='macro'),
        
        f1_train = f1_score(y_true=y_train_num, y_pred=pipe_preds_train, average='macro'),
        f1_test = f1_score(y_true=y_test_num, y_pred=pipe_preds_test, average='macro'),
    
        accuracy_train = accuracy_score(y_true=y_train_num, y_pred=pipe_preds_train),
        accuracy_test = accuracy_score(y_true=y_test_num, y_pred=pipe_preds_test)
    ))
results


100%|██████████████████████████████████████████| 9/9 [1:08:36<00:00, 457.37s/it]

CPU times: user 3h 20min 17s, sys: 37.8 s, total: 3h 20min 55s
Wall time: 1h 8min 36s


[{'params': {'n_estimators': 600, 'xgb_learning_rate': 0.2},
  'precision_train': 0.8602533579511242,
  'precision_test': 0.5978361276185351,
  'recall_train': 0.8475135214983179,
  'recall_test': 0.588493539987323,
  'f1_train': 0.8503720174538696,
  'f1_test': 0.5885371524470866,
  'accuracy_train': 0.8440852241831035,
  'accuracy_test': 0.5802194543297746},
 {'params': {'n_estimators': 600, 'xgb_learning_rate': 0.3},
  'precision_train': 0.9001055763896986,
  'precision_test': 0.6048107721191295,
  'recall_train': 0.8928411392053043,
  'recall_test': 0.5970870245784513,
  'f1_train': 0.8943356042084872,
  'f1_test': 0.5973396550949013,
  'accuracy_train': 0.8889218448761679,
  'accuracy_test': 0.5876334519572953},
 {'params': {'n_estimators': 600, 'xgb_learning_rate': 0.4},
  'precision_train': 0.9274101402726747,
  'precision_test': 0.6078354476196175,
  'recall_train': 0.9222275301428319,
  'recall_test': 0.5979508286083149,
  'f1_train': 0.9233662937579646,
  'f1_test': 0.5993820

In [ ]:
results = pd.DataFrame(results)
results.sort_values('accuracy_test', ascending=False).head(10).style.bar(vmin=0, vmax=1)

,params,precision_train,precision_test,recall_train,recall_test,f1_train,f1_test,accuracy_train,accuracy_test
2,"{'n_estimators': 600, 'xgb_learning_rate': 0.4}",0.927410,0.607835,0.922228,0.597951,0.923366,0.599382,0.918978,0.591934
7,"{'n_estimators': 800, 'xgb_learning_rate': 0.3}",0.928269,0.606927,0.923561,0.598282,0.924467,0.599298,0.920164,0.590451
5,"{'n_estimators': 700, 'xgb_learning_rate': 0.4}",0.941597,0.605946,0.938497,0.596146,0.938943,0.597887,0.935439,0.590006
4,"{'n_estimators': 700, 'xgb_learning_rate': 0.3}",0.916908,0.607280,0.910702,0.597497,0.912059,0.598907,0.907212,0.589858
8,"{'n_estimators': 800, 'xgb_learning_rate': 0.4}",0.950880,0.603013,0.948917,0.595077,0.949017,0.596288,0.945969,0.588820
6,"{'n_estimators': 800, 'xgb_learning_rate': 0.2}",0.888728,0.605992,0.879063,0.596220,0.881204,0.597030,0.875772,0.588227
1,"{'n_estimators': 600, 'xgb_learning_rate': 0.3}",0.900106,0.604811,0.892841,0.597087,0.894336,0.597340,0.888922,0.587633
3,"{'n_estimators': 700, 'xgb_learning_rate': 0.2}",0.875101,0.602057,0.863566,0.591748,0.866265,0.592442,0.860497,0.583926
0,"{'n_estimators': 600, 'xgb_learning_rate': 0.2}",0.860253,0.597836,0.847514,0.588494,0.850372,0.588537,0.844085,0.580219


## 5.4.  Evaluation on the Train Sample & the Test Sample

In [ ]:
print(classification_report(y_true=y_train_num, y_pred=tf_pred_train))
print('Balanced accuracy:', round(balanced_accuracy_score (y_true=y_train_num, y_pred=tf_pred_train), 3))

              precision    recall  f1-score   support

           0       0.99      0.94      0.96      2751
           1       0.95      0.90      0.92      4899
           2       0.85      0.97      0.91      3733
           3       0.94      0.93      0.94      5584
           4       0.99      0.95      0.97      3262

    accuracy                           0.94     20229
   macro avg       0.94      0.94      0.94     20229
weighted avg       0.94      0.94      0.94     20229

Balanced accuracy: 0.938


In [ ]:
ConfusionMatrixDisplay.from_predictions(y_true=y_train_num, y_pred=tf_pred_train)
plt.xticks(rotation=70)
plt.show()

In [ ]:
print(classification_report(y_true=y_test_num, y_pred=tf_pred_test))
print('Balanced accuracy:', round(balanced_accuracy_score (y_true=y_test_num, y_pred=tf_pred_test), 3))

              precision    recall  f1-score   support

           0       0.65      0.55      0.60       917
           1       0.52      0.49      0.50      1634
           2       0.63      0.77      0.69      1244
           3       0.54      0.58      0.56      1862
           4       0.69      0.60      0.64      1087

    accuracy                           0.59      6744
   macro avg       0.61      0.60      0.60      6744
weighted avg       0.59      0.59      0.59      6744

Balanced accuracy: 0.596


In [ ]:
ConfusionMatrixDisplay.from_predictions(y_true=y_test_num, y_pred=tf_pred_test)
plt.xticks(rotation=70)
plt.show()

# 6. Whole Sample training

## `X` & `y` 

**Note:** after choosing an optimal model on the `corona_train.csv` splitting base, let's implement it on the `corona_test.csv`!

In [ ]:
%%time
pipeline.fit(X.CleanTweet, y_num)

tf_pred = pipeline.predict(X.CleanTweet)

CPU times: user 25min 54s, sys: 4.96 s, total: 25min 59s
Wall time: 8min 49s


In [ ]:
print(classification_report(y_true=y_num, y_pred=tf_pred))
print('Balanced accuracy:', round(balanced_accuracy_score (y_true=y_num, y_pred=tf_pred), 3))

              precision    recall  f1-score   support

           0       0.96      0.90      0.93      3668
           1       0.91      0.85      0.88      6533
           2       0.80      0.94      0.86      4977
           3       0.89      0.89      0.89      7446
           4       0.97      0.92      0.94      4349

    accuracy                           0.90     26973
   macro avg       0.91      0.90      0.90     26973
weighted avg       0.90      0.90      0.90     26973

Balanced accuracy: 0.901


# 7. `test_data`

## 7.1. Dataset & Missing values

In [ ]:
test_data = pd.read_csv('corona_test.csv', encoding='ISO-8859-1', index_col=0)
test_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet
ID,,,,,
0,36853,81805,NaN,07-04-2020,https://t.co/2yMUPatIYU Wear a mask! Especiall...
1,37363,82315,NaN,08-04-2020,"In light of Covid-19, Canada's insurers are re..."
2,36506,81458,5280,07-04-2020,@ate_ted @SenTedCruz Supply &amp; Demand. COVI...
3,17051,62003,"Sydney, New South Wales",21-03-2020,Around 9 million people die of hunger and hung...
4,41636,86588,India,11-04-2020,Alarming but informative video everyone should...


In [ ]:
test_data.shape

(17982, 5)

In [ ]:
# percentage of missing data
(test_data.isna().sum() / len(test_data)).rename('percent of NA').to_frame().style.bar(vmax=1, vmin=0).format('{:.2%}')

## 7.2. Data types & Prepsocessing & Converting

In [ ]:
# converting to string
test_data.UserName = test_data.UserName.astype(str)
test_data.ScreenName = test_data.ScreenName.astype(str)
test_data.TweetAt = pd.to_datetime(test_data.TweetAt)

In [ ]:
test_data['month'] = test_data.TweetAt.dt.month
test_data['day'] = test_data.TweetAt.dt.day
test_data['dayofweek'] = test_data.TweetAt.dt.dayofweek
test_data['weekday'] = test_data.TweetAt.dt.weekday
test_data['tweetlength'] = test_data.OriginalTweet.str.len()

### `CleanTweet`

In [ ]:
# for checking 
test_data['CleanTweet'][5]

'disrupt global supplychain be felt across world thank covid pandem recent week demand groceri item deliveri those item surg coronaviru wyijfwyen'

In [ ]:
# cleaning
test_data['CleanTweet'] = test_data['OriginalTweet'].str.lower()

test_data['CleanTweet'] = test_data['CleanTweet'].str.replace("[^a-zA-Z#]", " ") # base
test_data['CleanTweet'] = test_data['CleanTweet'].apply(lambda x: x.replace('\n', ' ')) # base

test_data['CleanTweet'] = test_data['CleanTweet'].str.replace('[^A-Za-z0-9]+',' ', regex=True) # kaggle
test_data['CleanTweet'] = test_data['CleanTweet'].str.replace(r"http\S+", "", regex=True) # kaggle

test_data['CleanTweet'] = np.vectorize(remove_pattern)(test_data['CleanTweet'], "@[\w]*") # git
test_data['CleanTweet'] = test_data['CleanTweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3])) # git

# stemming
tokenized_tweets = test_data['CleanTweet'].apply(lambda x: x.split()) 
stemmer = PorterStemmer() 
tokenized_tweets = tokenized_tweets.apply(lambda x: [stemmer.stem(i) for i in x]) 

for i in range(len(tokenized_tweets)):
    tokenized_tweets[i] = ' '.join(tokenized_tweets[i])   
test_data['CleanTweet'] = tokenized_tweets

## 7.3. Prediction on `test_data`

In [ ]:
tf_pred_test_data = pipeline.predict(test_data.CleanTweet)

In [ ]:
tf_pred_test_data

array([4, 3, 1, ..., 2, 4, 3])

**Note:** before submitting, let's convert back 5-classes numeric labels to the text ones.

In [ ]:
submission = pd.DataFrame(data = tf_pred_test_data, index = test_data.index,
             columns=['Sentiment']).replace({0: 'Extremely Negative',
                   1: 'Negative',
                   2: 'Neutral',
                   3: 'Positive',
                   4: 'Extremely Positive'})
submission

,Sentiment
ID,
0,Extremely Positive
1,Positive
2,Negative
3,Extremely Negative
4,Extremely Positive
...,...
17977,Positive
17978,Neutral
17979,Neutral


In [ ]:
submission.to_csv('submission.csv')